# Gold Prediction Project - Data Cleaning

## Introduction
This notebook will show the data cleaning process for my Gold Predictor project. So far I have collated data for a variety of macroeconomic factors, as well as the gold price itself. Before exploring the data, there are already a few goals I have going into the data cleaning process:

- The datasets are not in the same format or shape, and will require some adjusting before I can merge them together.
- The GDP data is quarterly. I will need to fill the missing values.
- The final dataset should be in the following format:
    - | date | gold_price | gold_price_nextm | sp500 | interest_rate | gdp | cpi | dxy |
      | :-: | :-: | :-: | :-: | :-: | :-: | :-: | :-: |

Descriptions for each attibute:
- ***date***: Represented as DD/MM/YYYY. Data assumed to be collected on the first day of each month.
- ***gold_price***: (USD) Price of gold on first day of month.
- ***gold_price_nextm***: (USD) Price of gold of following month. This will be constucted as I am assuming that the macroeconomic factors will show triggers of the gold price action of the following month.
- ***sp500***: Value of the S&P500 index.
- ***interest_rate***: US Federal interest rate.
- ***gdp***: Gross Domestic Product (**GDP**) of US.
- ***cpi***: Consumer Price Index (**CPI**) of US.
- ***dxy***: Index representing the comparative strength of **USD** to other currencies.

In [1]:
import pandas as pd
import numpy as np

## Gold Data Cleaning
First I will start by cleaning the gold pricing data.

In [2]:
# Read in data to pandas DataFrame
gold_df = pd.read_csv("../data/gold_data.csv")
gold_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,2/01/2025,"2,898.80","2,838.55","2,910.59","2,802.24",865.43K,2.25%
1,1/01/2025,"2,835.00","2,652.00","2,862.90","2,638.90",386.65K,6.85%
2,12/01/2024,"2,653.30","2,673.80","2,761.30","2,596.70",2.92M,-1.03%
3,11/01/2024,"2,681.00","2,766.90","2,782.80","2,560.80",360.31K,-2.92%
4,10/01/2024,"2,761.60","2,656.10","2,813.10","2,618.80",3.63M,4.29%


The only columns I am interested in, are the *date* and *price* columns. Lets drop the others from the dataset.

In [3]:
cols_to_drop = ["Open", "High", "Low", "Vol.", "Change %"]
gold_df.drop(columns=cols_to_drop, inplace=True)

gold_df.head()

,Date,Price
0,2/01/2025,"2,898.80"
1,1/01/2025,"2,835.00"
2,12/01/2024,"2,653.30"
3,11/01/2024,"2,681.00"
4,10/01/2024,"2,761.60"


In [4]:
gold_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    420 non-null    object
 1   Price   420 non-null    object
dtypes: object(2)
memory usage: 6.7+ KB


There are no missing values on first inspection.

Let's change the *Date* column into a datetime object, so it is easier to handle when we analyse. I'll also change the datatype of *Price* to **float**, which will require removing the commas. 

In [5]:
gold_df["Date"] = pd.to_datetime(gold_df["Date"], format="%m/%d/%Y")
gold_df["Price"] = gold_df["Price"].str.replace(",", "").astype("float")

gold_df.head()

,Date,Price
0,2025-02-01,2898.8
1,2025-01-01,2835.0
2,2024-12-01,2653.3
3,2024-11-01,2681.0
4,2024-10-01,2761.6


I will change the column names to lowercase as a personal preference, and sort the *date* column in ascending order. I'll also swap *price* to *gold_price*.

In [6]:
gold_df["date"] = gold_df["Date"]
gold_df["gold_price"] = gold_df["Price"]

gold_df.drop(columns=["Date", "Price"], inplace=True)

In [7]:
gold_df.sort_values(by="date", ascending=True, ignore_index=True)

,date,gold_price
0,1990-03-01,375.0
1,1990-04-01,371.4
2,1990-05-01,368.6
3,1990-06-01,359.9
4,1990-07-01,379.2
...,...,...
415,2024-10-01,2761.6
416,2024-11-01,2681.0
417,2024-12-01,2653.3
418,2025-01-01,2835.0


I'm happy with the state of the gold dataframe. Let's move on to the CPI data.

## CPI Data Cleaning

Now I will clean the CPI data and ensure it is in the same format as the gold dataframe.

Let's read the data in, and have a look at it's shape.

In [8]:
cpi_df = pd.read_csv("../data/cpi_data.csv")

cpi_df.head()

,Year,Jan,Feb,Mar,Apr,May,June,July,Aug,Sep,Oct,Nov,Dec,Avg,Dec-Dec,Avg-Avg
0,1913,9.8,9.8,9.8,9.8,9.7,9.8,9.9,9.9,10.0,10.0,10.1,10.0,9.9,–,–
1,1914,10.0,9.9,9.9,9.8,9.9,9.9,10.0,10.2,10.2,10.1,10.2,10.1,10.0,1.0,1.0
2,1915,10.1,10.0,9.9,10.0,10.1,10.1,10.1,10.1,10.1,10.2,10.3,10.3,10.1,2.0,1.0
3,1916,10.4,10.4,10.5,10.6,10.7,10.8,10.8,10.9,11.1,11.3,11.5,11.6,10.9,12.6,7.9
4,1917,11.7,12.0,12.0,12.6,12.8,13.0,12.8,13.0,13.3,13.5,13.5,13.7,12.8,18.1,17.4


In [9]:
cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Year     113 non-null    int64  
 1   Jan      113 non-null    float64
 2   Feb      113 non-null    object 
 3   Mar      112 non-null    float64
 4   Apr      112 non-null    float64
 5   May      112 non-null    float64
 6   June     112 non-null    float64
 7   July     112 non-null    float64
 8   Aug      112 non-null    float64
 9   Sep      112 non-null    float64
 10  Oct      112 non-null    float64
 11  Nov      112 non-null    float64
 12  Dec      112 non-null    float64
 13  Avg      112 non-null    float64
 14  Dec-Dec  112 non-null    object 
 15  Avg-Avg  112 non-null    object 
dtypes: float64(12), int64(1), object(3)
memory usage: 14.3+ KB


I'll first drop the "average" columns, and convert the data in the *Jan* column to **float**.

After that, I'll need to transform the dataframe so the month attributes are represented in a column *month*, so that each row is a single observation.

In [10]:
# Drop columns
cols_to_drop = ["Avg", "Dec-Dec", "Avg-Avg"]
cpi_df.drop(columns=cols_to_drop, inplace=True)

In [11]:
# "Melt" to reshape to ['year', 'month]
cpi_df_melted = pd.melt(
    cpi_df,
    id_vars=["Year"],
    var_name="month",
    value_name="cpi"
)

month_map = {
    'Jan': '01', 'Feb': '02', 'Mar': '03',
    'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09',
    'Oct': '10', 'Nov': '11', 'Dec': '12'
}

cpi_df_melted["month"] = cpi_df_melted["month"].map(month_map)

cpi_df_melted["date"] = pd.to_datetime(
    cpi_df_melted["Year"].astype(str)+
    "/"+
    cpi_df_melted["month"]+
    "/"+
    "01",
    format="%Y/%m/%d"
)

cpi_df = cpi_df_melted[["date", "cpi"]]

In [12]:
# Recheck shape of df
cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1130 non-null   datetime64[ns]
 1   cpi     1346 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 21.3+ KB


In [13]:
# cpi_df["cpi"] = cpi_df["cpi"].str.replace(",", "").astype("float")


I tried to run the above commented code but ran into this error: `ValueError: could not convert string to float: 'Avail.Feb.12'`.

One or some of the values in the *cpi* column must not be numerical. Let's find out which rows are affected.

In [14]:
cpi_df["cpi"] = pd.to_numeric(cpi_df["cpi"], errors="coerce")

non_numerical_idx = cpi_df[cpi_df["cpi"].isna()]

C:\Users\keans\AppData\Local\Temp\ipykernel_10840\600742422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cpi_df["cpi"] = pd.to_numeric(cpi_df["cpi"], errors="coerce")


In [15]:
non_numerical_idx

,date,cpi
225,2025-02-01,NaN
338,2025-03-01,NaN
451,2025-04-01,NaN
564,2025-05-01,NaN
677,NaT,NaN
790,NaT,NaN
903,2025-08-01,NaN
1016,2025-09-01,NaN
1129,2025-10-01,NaN
1242,2025-11-01,NaN


In [16]:
# Remove rows in non_numerical_idx from cpi_df
idx = non_numerical_idx.index

cpi_df = cpi_df.drop(idx).reset_index(drop=True)

In [17]:
cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1345 entries, 0 to 1344
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1121 non-null   datetime64[ns]
 1   cpi     1345 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 21.1 KB


In [18]:
cpi_df

,date,cpi
0,1913-01-01,9.800
1,1914-01-01,10.000
2,1915-01-01,10.100
3,1916-01-01,10.400
4,1917-01-01,11.700
...,...,...
1340,2020-12-01,260.474
1341,2021-12-01,278.802
1342,2022-12-01,296.797
1343,2023-12-01,306.746


I'm happy with the state of the CPI data for now. Let's move on to the DXY data.

## DXY Data Cleaning

I will first read dxy_data.csv in and take a look at the shape of the data.

In [19]:
dxy_data = pd.read_csv("../data/dxy_data.csv")

dxy_data.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,01/02/2025,108.29,109.70,109.88,107.30,0.00K,-0.07%
1,01/01/2025,108.37,108.53,110.18,106.97,0.02K,-0.11%
2,01/12/2024,108.49,105.83,108.58,105.42,NaN,2.60%
3,01/11/2024,105.74,103.86,108.07,103.37,NaN,1.69%
4,01/10/2024,103.98,100.74,104.64,100.70,NaN,3.17%


We are only interested in the *date* and *price* columns. Let's first drop the rest, and then convert the Price label to *dxy*.

In [20]:
dxy_data.drop(columns=["Open", "High", "Low", "Vol.", "Change %"], inplace=True)
dxy_data.rename(columns={"Date": "date", "Price": "dxy"}, inplace=True)

dxy_data.head()

,date,dxy
0,01/02/2025,108.29
1,01/01/2025,108.37
2,01/12/2024,108.49
3,01/11/2024,105.74
4,01/10/2024,103.98


In [21]:
dxy_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    422 non-null    object 
 1   dxy     422 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.7+ KB


Now, I will convert the dataatype of the *date* column to a pandas datetime object.

In [22]:
dxy_data["date"] = pd.to_datetime(dxy_data["date"], format="%d/%m/%Y")
dxy_data.head()

,date,dxy
0,2025-02-01,108.29
1,2025-01-01,108.37
2,2024-12-01,108.49
3,2024-11-01,105.74
4,2024-10-01,103.98


The DXY data is in good shape for now. I will move on to the S&P500 data.

## S&P500 Data Cleaning

In [23]:
sp500_data = pd.read_csv("../data/sp500_data.csv")
sp500_data.head()

,Date,Open,High,Low,CloseClose price adjusted for splits.,Adj CloseAdjusted close price adjusted for splits and dividend and/or capital gain distributions.,Volume
0,"Feb 1, 2025","5,969.65","6,101.28","5,923.93","6,051.97","6,051.97","36,989,240,000"
1,"Jan 1, 2025","5,903.26","6,128.18","5,773.31","6,040.53","6,040.53","88,639,380,000"
2,"Dec 1, 2024","6,040.11","6,099.97","5,832.30","5,881.63","5,881.63","86,064,900,000"
3,"Nov 1, 2024","5,723.22","6,044.17","5,696.51","6,032.38","6,032.38","84,101,980,000"
4,"Oct 1, 2024","5,757.73","5,878.46","5,674.00","5,705.45","5,705.45","82,412,430,000"


As before, I will drop columns that I don't need, keeping the *date* and *close* columns.

In [24]:
sp500_data.drop(columns=["Open", "High", "Low", "Adj CloseAdjusted close price adjusted for splits and dividend and/or capital gain distributions.", "Volume"], inplace=True)
sp500_data.rename(columns={"Date": "date", "CloseClose price adjusted for splits.": "sp500"}, inplace=True)

sp500_data.head()

,date,sp500
0,"Feb 1, 2025","6,051.97"
1,"Jan 1, 2025","6,040.53"
2,"Dec 1, 2024","5,881.63"
3,"Nov 1, 2024","6,032.38"
4,"Oct 1, 2024","5,705.45"


Next, I will need to reformat the date column to be a pandas datetime object. I'll split the date string and then can convert it from there.

In [26]:
sp500_data['month'] = sp500_data['date'].str.split(' ').str[0]
sp500_data['day'] = sp500_data['date'].str.split(' ').str[1]
sp500_data['year'] = sp500_data['date'].str.split(' ').str[2]


In [27]:
sp500_data.head()

,date,sp500,month,day,year
0,"Feb 1, 2025","6,051.97",Feb,"1,",2025
1,"Jan 1, 2025","6,040.53",Jan,"1,",2025
2,"Dec 1, 2024","5,881.63",Dec,"1,",2024
3,"Nov 1, 2024","6,032.38",Nov,"1,",2024
4,"Oct 1, 2024","5,705.45",Oct,"1,",2024


In [ ]:
# Reuse month_map from earlier to convert to datetime
